In [1]:
import os
import sys


os.chdir("/teamspace/studios/this_studio/nurse-helper-chatbot")


sys.path.append(os.getcwd())

In [6]:
from llm import llm
from graph import graph
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from utils import get_session_id
from tools.hospital import hospital_cypher_qa
from tools.patient import patient_cypher_qa
from tools.physician import physician_cypher_qa
from tools.checkup import checkup_cypher_qa
from tools.review import get_review
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler
import streamlit as st 
from langchain_core.runnables import RunnableConfig  


@tool
def explore_hospital(question: str) -> str:
    """Provide information about hospital-related questions using Cypher."""
    return hospital_cypher_qa.invoke(question)

@tool
def explore_patient(question: str) -> str:
    """Provide information about patient-related questions using Cypher."""
    return patient_cypher_qa.invoke(question)

@tool
def explore_checkup(question: str) -> str:
    """Provide information about checkup-related questions using Cypher."""
    return checkup_cypher_qa.invoke(question)

@tool
def explore_physician(question: str) -> str:
    """Provide information about physician-related questions using Cypher."""
    return physician_cypher_qa.invoke(question)
    
@tool
def explore_review(question: str) -> str:
    """Provide information about reviews."""
    return get_review(question)




tools = [
    explore_hospital,
    explore_patient,
    explore_checkup,
    explore_review,
    explore_physician,
]

system_message = SystemMessage(content="""

Thought: What action and insight you need from the context?
Cypher_query: What cypher query you use to retrieve the information?
Tool_used: What tool you use to retrieve the information?
Relationship: Draw the relationship between the entities in the context in the markdown format.
Response: list the details in bulletpoint to make it more readable

Use the following format to provide a response:
Thought: \n
Cypher_query: \n
Tool_used: \n
Relationship: \n
Response: \n
""")

memory = MemorySaver()

langgraph_agent_executor = create_react_agent(llm, tools, state_modifier=system_message, checkpointer=memory)

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

def generate_response(query: str):
    """
    Invoke the LangGraph agent with a user query and return only the output content.
    
    Args:
        query (str): The user query.
    
    Returns:
        str: The output content from the agent's response.
    """ 

    thread_id = "test-thread"    
    session_id = get_session_id()  
    message_history = get_memory(session_id)

    # Generate a unique checkpoint namespace and ID
    checkpoint_ns = "namespace"  # Replace with your logic for namespace
    checkpoint_id = f"{session_id}_{thread_id}"  # Example of generating a unique checkpoint ID

    config = RunnableConfig(
        configurable={
            "session_id": session_id,
            "thread_id": thread_id,
            "checkpoint_ns": checkpoint_ns,
            "checkpoint_id": checkpoint_id,
        }
    )

    messages = langgraph_agent_executor.invoke({
        "messages": [HumanMessage(content=query)],
        "chat_history": message_history, 
        "config": config
    })

    if "messages" in messages:
        for message in messages["messages"]:
            if isinstance(message, ToolMessage):
                with st.expander("Tool Message", expanded=False):  
                    st.write("Content:", message.content)  

    output_content = messages["messages"][-1].content if "messages" in messages else "No response generated."
    
    return output_content


In [7]:
# Import necessary modules
from llm import llm  # Ensure this is your language model
from tools.hospital import hospital_cypher_qa
from tools.patient import patient_cypher_qa
from tools.checkup import checkup_cypher_qa
from tools.review import get_review  # Ensure this is the correct import for your review tool
from tools.physician import physician_cypher_qa  # Ensure this is the correct import for your physician tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
from utils import get_session_id

# Define your tools using the @tool decorator for LangGraph integration
@tool
def explore_hospital(question: str) -> str:
    """Provide information about hospital-related questions using Cypher."""
    return hospital_cypher_qa.invoke(question)

@tool
def explore_patient(question: str) -> str:
    """Provide information about patient-related questions using Cypher."""
    return patient_cypher_qa.invoke(question)

@tool
def explore_checkup(question: str) -> str:
    """Provide information about checkup-related questions using Cypher."""
    return checkup_cypher_qa.invoke(question)

@tool
def explore_review(question: str) -> str:
    """Provide information about reviews using Cypher."""
    return get_review(question)
           
@tool
def explore_physician(question: str) -> str:
    """Provide information about physician-related questions using Cypher."""
    return physician_cypher_qa.invoke(question)

# List of tools for the LangGraph agent
tools = [
    explore_hospital,
    explore_patient,
    explore_checkup,
    explore_review,
    explore_physician,
]

# Create the LangGraph agent executor with the language model and tools
langgraph_agent_executor = create_react_agent(llm, tools)




In [8]:
def generate_response(query: str):
    """
    Invoke the LangGraph agent with a user query and return the response.
    """
    # Prepare messages for the agent (using HumanMessage)
    messages = langgraph_agent_executor.invoke({"messages": [HumanMessage(content=query)]})

    # Extract the last AI message content from the response
    output_content = messages["messages"][-1].content if "messages" in messages else "No response generated."

    # Return structured output
    return {
        "input": query,
        "output": output_content,
    }

In [5]:
def generate_response(query: str):
    """
    Invoke the LangGraph agent with a user query and return all messages.
    
    Args:
        query (str): The user query.
    
    Returns:
        list: A list of messages from the conversation including user and agent responses.
    """
    # Retrieve memory (message history) for this session
    session_id = get_session_id()  # Get or create a unique session ID
    message_history = get_memory(session_id)

    # Prepare messages for the agent (using HumanMessage)
    messages = langgraph_agent_executor.invoke({
        "messages": [HumanMessage(content=query)],
        "chat_history": message_history  # Pass the message history to the agent executor
    })

    # Return all messages from the response
    return messages["messages"] if "messages" in messages else []

# Example usage:
query = "What are the reviews for nurse service quality care?"
response = generate_response(query)
print(response)  # Display the input query and corresponding output from the agent

# You can also test other queries as needed.



2025-01-02 04:13:22.530 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 04:13:22.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


[HumanMessage(content='What are the reviews for nurse service quality care?', additional_kwargs={}, response_metadata={}, id='0f442167-5022-4e4b-9e7f-b8e3f42c372a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_G5hvSMTkirnXDfCjPwh9J9Eh', 'function': {'arguments': '{"question":"nurse service quality care"}', 'name': 'explore_review'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 155, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e36ed1c6-afc1-42aa-8598-3a9532dddbc1-0', tool_calls=[{'name': 'explore_review', 'args': {'question': 'nurse service quality care'}, 'id': 'call_G5hvSMTkir

In [5]:
def generate_response(query: str):
    """
    Invoke the LangGraph agent with a user query and return all messages.
    
    Args:
        query (str): The user query.
    
    Returns:
        list: A list of messages from the conversation including user and agent responses.
    """
    # Retrieve memory (message history) for this session
    session_id = get_session_id()  # Get or create a unique session ID
    message_history = get_memory(session_id)

    # Prepare messages for the agent (using HumanMessage)
    messages = langgraph_agent_executor.invoke({
        "messages": [HumanMessage(content=query)],
        "chat_history": message_history  # Pass the message history to the agent executor
    })

    # Return all messages from the response
    return messages["messages"] if "messages" in messages else []
# You can test other queries as well:
query2 = "What checkups attended by Dr. Aisyah binti Kamaruddin include patient details?"
response2 = generate_response(query2)
print(response2)

2025-01-02 10:55:32.508 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 10:55:32.509 Session state does not function when running a script without `streamlit run`
2025-01-02 10:55:32.510 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 10:55:32.511 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 10:55:32.511 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


NameError: name 'get_memory' is not defined

In [9]:
## List doctors specializing in Cardiology or Orthopedics at Penang General Hospital and Penang Adventist Hospital
query2 = "List doctors specializing in Cardiology or Orthopedics at Penang General Hospital and Penang Adventist Hospital"
response2 = generate_response(query2)
print(response2)



> Entering new GraphCypherQAChain chain...

> Entering new GraphCypherQAChain chain...



> Entering new GraphCypherQAChain chain...


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE p.Specialization = "Cardiology" AND h.Hospital_Name = "Penang Adventist Hospital"
RETURN p.Physician_Name AS Name, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber;

Generated Cypher:

MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE p.Specialization = "Cardiology" AND h.Hospital_Name = "Penang General Hospital"
RETURN p.Physician_Name AS Name, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber;

Generated Cypher:

MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE p.Specialization = "Orthopedics" AND h.Hospital_Name = "Penang General Hospital"
RETURN p.Physician_Name AS Name, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact

In [2]:
from llm import llm
from graph import graph
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from utils import get_session_id
from tools.hospital import hospital_cypher_qa
from tools.patient import patient_cypher_qa
from tools.physician import physician_cypher_qa
from tools.checkup import checkup_cypher_qa
from tools.review import get_review
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver
import streamlit as st 
from langchain_core.runnables import RunnableConfig  
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

@tool
def explore_hospital(question: str) -> str:
    """Provide information about hospital-related questions using Cypher."""
    return hospital_cypher_qa.invoke(question)

@tool
def explore_patient(question: str) -> str:
    """Provide information about patient-related questions using Cypher."""
    return patient_cypher_qa.invoke(question)

@tool
def explore_checkup(question: str) -> str:
    """Provide information about checkup-related questions using Cypher."""
    return checkup_cypher_qa.invoke(question)

@tool
def explore_physician(question: str) -> str:
    """Provide information about physician-related questions using Cypher."""
    return physician_cypher_qa.invoke(question)
    
@tool
def explore_review(question: str) -> str:
    """Provide information about reviews using Cypher."""
    return get_review(question)




tools = [
    explore_hospital,
    explore_patient,
    explore_checkup,
    explore_review,
    explore_physician,
]

system_message = SystemMessage(content="""

Thought: What action and insight you need from the context?
Cypher_query: What cypher query you use to retrieve the information?
Tool_used: What tool you use to retrieve the information?
Relationship: Draw the relationship between the entities in the context in the markdown format.
Response: list the details in bulletpoint to make it more readable

Use the following format to provide a response:
Thought: \n
Cypher_query: \n
Tool_used: \n
Relationship: \n
Response: \n
""")



langgraph_agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)




/teamspace/studios/this_studio/nurse-helper-chatbot/graph.py:7: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [5]:
def generate_response(query: str):
    """
    Invoke the LangGraph agent with a user query and print all messages.
    
    Args:
        query (str): The user query.
    
    Returns:
        str: The output content from the agent's response.
    """
    # Retrieve memory (message history) for this session
    session_id = get_session_id()  # Get or create a unique session ID
    message_history = get_memory(session_id)

    # Prepare messages for the agent (using HumanMessage)
    messages = langgraph_agent_executor.invoke({
        "messages": [HumanMessage(content=query)],
        "chat_history": message_history  # Pass the message history to the agent executor
    })

    print("Process:", messages)

    # Extract the last AI message content from the response
    output_content = messages["messages"][-1].content if "messages" in messages else "No response generated."

    # Return only the output content
    return output_content


In [3]:
def generate_response(query: str):
    """
    Invoke the LangGraph agent with a user query and return only the output content.
    
    Args:
        query (str): The user query.
    
    Returns:
        str: The output content from the agent's response.
    """ 

    session_id = get_session_id()  # Get or create a unique session ID
    
    # Generate unique identifiers for thread and checkpointing
    thread_id = get_session_id()  # Generate a unique thread ID
    checkpoint_ns = "namespace"  # Set a default namespace or generate as needed
    checkpoint_id = get_session_id()  # Generate a unique checkpoint ID

    message_history = get_memory(session_id)

    config = RunnableConfig(
        configurable={
            "session_id": session_id,
            "thread_id": thread_id,
            "checkpoint_ns": checkpoint_ns,
            "checkpoint_id": checkpoint_id,
        }
    )

    # Prepare messages for the agent (using HumanMessage)
    messages = langgraph_agent_executor.invoke({
        "messages": [HumanMessage(content=query)],
        "chat_history": message_history, 
        "config": config  # Pass the configuration to the agent executor
    })

    # Print only ToolMessages in Streamlit expander for debugging purposes
    if "messages" in messages:
        for message in messages["messages"]:
            if isinstance(message, ToolMessage):
                with st.expander("Tool Message", expanded=False):  
                    st.write("Content:", message.content)  

    # Extract the last AI message content from the response
    output_content = messages["messages"][-1].content if "messages" in messages else "No response generated."

    # Return only the output content
    return output_content


In [26]:
## List doctors specializing in Cardiology or Orthopedics at Penang General Hospital and Penang Adventist Hospital
query2 = "List doctors specializing in Cardiology or Orthopedics at Penang General Hospital and Penang Adventist Hospital"
response2 = generate_response(query2)
print(response2)

2025-01-05 14:40:38.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:38.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:38.242 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:38.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:38.244 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:38.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.




> Entering new GraphCypherQAChain chain...


> Entering new GraphCypherQAChain chain...


> Entering new GraphCypherQAChain chain...


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE p.Specialization = "Orthopedics" AND h.Hospital_Name = "Penang General Hospital"
RETURN p.Physician_Name AS Name, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber;

Generated Cypher:

MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE p.Specialization = "Cardiology" AND h.Hospital_Name = "Penang General Hospital"
RETURN p.Physician_Name AS Name, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber;

Generated Cypher:

MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE h.Hospital_Name = "Penang Adventist Hospital" AND 
      p.Specialization = "Cardiology"
RETURN p.Physician_Name AS Name, 
       p.License_Number AS LicenseNumber, 
       p.Physician_

2025-01-05 14:40:54.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:54.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:54.946 
  command:

    streamlit run /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-05 14:40:54.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:54.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:54.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:54.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 14:40:54.950 Thread 'MainThread': missing ScriptRunContext! This warning can 

Here are the doctors specializing in Cardiology and Orthopedics at Penang General Hospital and Penang Adventist Hospital:

### Penang General Hospital
**Cardiology Specialists:**
1. Dr. Letchumi a/p Balakrishnan
   - License Number: 48125
   - Contact: 03-5129887
2. Dr. Sharifah Nora binti Syed Ahmad
   - License Number: 82414
   - Contact: 03-7786393
3. Dr. Loo Yen Fang
   - License Number: 10122
   - Contact: 03-1836810
4. Dr. Cheah Wai Lum
   - License Number: 98432
   - Contact: 03-5647382

**Orthopedics Specialists:**
1. Dr. Balakrishnan Menon
   - License Number: 91109
   - Contact: 03-6459865
2. Dr. Ng Qian Yi
   - License Number: 22293
   - Contact: 03-4145827
3. Dr. Chee Seok Wing
   - License Number: 44822
   - Contact: 03-9925967
4. Dr. Low Hui Ting
   - License Number: 66534
   - Contact: 03-4895732

### Penang Adventist Hospital
**Cardiology Specialists:**
1. Dr. Siti Hasmah binti Mokhtar
   - License Number: 61601
   - Contact: 03-3985470

**Orthopedics Specialists:**
1. 

In [15]:
# What are the details of checkups conducted in room A314?
query2 = "What are the details of checkups conducted in room A314?"
response2 = generate_response(query2)
print(response2)

2025-01-02 15:30:48.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 15:30:48.909 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.




> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (c:checkup)
WHERE c.Room_Number = "A314"
RETURN c.Checkup_ID AS CheckupID, 
       c.Admission_Type AS AdmissionType, 
       c.Checkup_Status AS CheckupStatus, 
       c.Diagnosis AS Diagnosis, 
       c.Medication AS Medication;

Full Context:
[{'CheckupID': 1, 'AdmissionType': 'Urgent', 'CheckupStatus': 'Discharged', 'Diagnosis': 'Acute myocardial infarction', 'Medication': 'Aspirin'}]

> Finished chain.
Process: {'messages': [HumanMessage(content='What are the details of checkups conducted in room A314?', additional_kwargs={}, response_metadata={}, id='2b35047b-a98a-42c2-bde0-c38253a74266'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_spr6nvLcdiuNDppZky6PDEND', 'function': {'arguments': '{"question":"What are the details of checkups conducted in room A314?"}', 'name': 'explore_checkup'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'p

In [17]:
query2 = "Retrieve checkups for Dr. Ng Yee Siang that involve emergencies or abnormal test results."
response2 = generate_response(query2)
print(response2)

2025-01-02 15:31:42.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 15:31:42.774 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.




> Entering new GraphCypherQAChain chain...


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)
WHERE c.Admission_Type = "Emergency" AND ph.Physician_Name = "Dr. Ng Yee Siang"
RETURN c.Checkup_ID AS CheckupID, 
       c.Admission_Date AS AdmissionDate, 
       c.Discharge_Date AS DischargeDate, 
       c.Diagnosis AS Diagnosis, 
       c.Medication AS Medication, 
       p.Name AS PatientName;

Generated Cypher:

MATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)
WHERE ph.Physician_Name = "Dr. Ng Yee Siang" AND c.Test_Results = "Abnormal"
RETURN c.Checkup_ID AS CheckupID, 
       c.Admission_Type AS AdmissionType, 
       c.Checkup_Status AS CheckupStatus, 
       c.Diagnosis AS Diagnosis, 
       c.Treatment AS Treatment, 
       c.Medication AS Medication, 
       p.Name AS PatientName;

Full Context:
[{'CheckupID': 190, 'AdmissionDate': '8/2/2024', 'DischargeDate': None, 'Diagnosis': 'Arthritis

In [6]:
# You can test other queries as well:
query2 = " can u give me the patient info with Identity Number 880519-05-9016?"
response2 = generate_response(query2)
print(response2)

2025-01-02 14:42:26.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 14:42:26.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.




> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:patient {Identity_number: "880519-05-9016"})
RETURN properties(p) AS PatientProperties; 

Full Context:
[{'PatientProperties': {'Name': 'Mohd Faizal', 'Identity_number': '880519-05-9016', 'Age': 36, 'Allergy': 'None', 'Patient_Contact': '016-1531446', 'Emergency_number': '013-9656371', 'Gender': 'Male', 'Patient_ID': 20, 'Blood_Type': 'O+', 'Chronic_condition': 'Lung Infections'}}]

> Finished chain.
Process: {'messages': [HumanMessage(content=' can u give me the patient info with Identity Number 880519-05-9016?', additional_kwargs={}, response_metadata={}, id='884fe6e9-d014-49b2-a938-14ef455f6fbe'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OD6u6R3jn6ScQreRIbpfP0JF', 'function': {'arguments': '{"question":"patient info with Identity Number 880519-05-9016"}', 'name': 'explore_patient'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'p

In [5]:
query2 = "Find physicians specializing in Psychiatry or Endocrinology at Penang Adventist Hospital."
response2 = generate_response(query2)
print(response2)

2025-01-02 15:19:34.588 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 15:19:34.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 15:19:34.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-02 15:19:34.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.




> Entering new GraphCypherQAChain chain...


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE h.Hospital_Name = "Penang Adventist Hospital" AND 
      p.Specialization = "Endocrinology"
RETURN p.Physician_Name AS Name, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber;

Generated Cypher:

MATCH (p:physician)<-[:recruit]-(h:hospital)
WHERE p.Specialization = "Psychiatry" AND h.Hospital_Name = "Penang Adventist Hospital"
RETURN p.Physician_Name AS Name, 
       p.License_Number AS LicenseNumber, 
       p.Physician_Contact AS PhysicianContactNumber;

Full Context:
[{'Name': 'Dr. Pooja Desai', 'LicenseNumber': 91410, 'PhysicianContactNumber': '03-4635181'}, {'Name': 'Dr. Goh Li Ping', 'LicenseNumber': 77564, 'PhysicianContactNumber': '03-3459721'}]

> Finished chain.
Full Context:
[{'Name': 'Dr. Mohd Nazri bin Shafie', 'LicenseNumber': 26840, 'PhysicianContactNumber': '03-91321

In [21]:
from llm import llm
from graph import graph
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from utils import get_session_id
from tools.hospital import hospital_cypher_qa
from tools.patient import patient_cypher_qa
from tools.physician import physician_cypher_qa
from tools.review import get_review
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler
import streamlit as st 
from langchain_core.runnables import RunnableConfig  


@tool
def explore_hospital(question: str) -> str:
    """Provide information about patient questions using Cypher and context from GraphCypherQAChain"""
    return hospital_cypher_qa.invoke(question)

@tool
def explore_patient(question: str) -> str:
    """Provide information about patient questions using Cypher and context from GraphCypherQAChain."""
    return patient_cypher_qa.invoke(question)

@tool
def explore_review(question: str) -> str:
    """Provide information about checkup questions using Cypher and context from GraphCypherQAChain."""
    return get_review.invoke(question)




tools = [explore_hospital, explore_patient, explore_review]


system_message = SystemMessage(content="""
You are a nursing expert providing information about nursing care.
Be as helpful as possible and return as much information as possible.
Only answer questions related to checkups, hospitals, psyhicians or patients.

You must rely solely on the information provided in the context and the tools available to you. 
Do not use any pre-trained knowledge.

You **must** use the tools provided for every relevant question. 
Do not decline to use a tool, as it is essential for generating accurate responses.

When generating responses:
Use the actual Cypher examples from the `GraphCypherQAChain` to construct accurate queries.
Validate the information retrieved from the tools before providing a final answer.
If the information is insufficient, indicate that further clarification is needed.

You may list the thoughts, tools used, and the response generated in the output.
""")


memory = MemorySaver()

langgraph_agent_executor = create_react_agent(llm, tools, state_modifier=system_message, checkpointer=memory)

# Define the memory function
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

# Create the agent executor
agent_executor = RunnableWithMessageHistory(
    langgraph_agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)




In [14]:
import streamlit as st
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler

def generate_response(user_input):
    """
    Create a handler that calls the Conversational agent
    and streams responses to be printed in the console.
    Returns the final output content from the last AIMessage after invocation.
    """
    # Define the messages structure using HumanMessage
    messages = [HumanMessage(content=user_input)]
    
    # Get the session ID and thread ID
    session_id = get_session_id()  # Assuming this generates a unique session ID
    thread_id = get_session_id()    # You can use the same function or create a new one for thread ID

    # Create a RunnableConfig instance
    config = RunnableConfig(
        configurable={
            "session_id": session_id,
            "thread_id": thread_id
        }
    )

    # Initialize the StreamlitCallbackHandler
    st_callback = StreamlitCallbackHandler(st.container())

    # Initialize a variable to hold the final output content
    final_output_content = ""

    # Prepare the input for the invoke method
    input_data = {
        "messages": messages,
        "input": user_input  # Ensure the input key is included
    }

    # Invoke the agent with the messages and configuration, including the callback
    response = langgraph_agent_executor.invoke(input_data, config, callbacks=[st_callback])
    

    # Check if the response contains messages
    if 'messages' in response:
        for message in response['messages']:
            if isinstance(message, AIMessage):
                # Update the final output content with the latest AIMessage content
                final_output_content = message.content  # Store the latest AIMessage content


    # Return the final output content
    return final_output_content




In [6]:
query = "What are the reviews for nurse service quality care?"
    
response = generate_response(query)

    
response

2025-01-05 17:55:08.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-05 17:55:08.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Process: {'messages': [HumanMessage(content='What are the reviews for nurse service quality care?', additional_kwargs={}, response_metadata={}, id='1ad6771e-9f73-4422-8a4d-e971ae627808'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ao5ifXTJ3n4o8RROhtbcaCen', 'function': {'arguments': '{"question":"nurse service quality care"}', 'name': 'explore_review'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 252, 'total_tokens': 272, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3062ec15-6062-4ee9-9f60-faca6aa87160-0', tool_calls=[{'name': 'explore_review', 'args': {'question': 'nurse service quality care'}, 

'Thought: I need to gather reviews specifically related to nurse service quality care in healthcare settings.\n\nCypher_query: None, as the review data has already been retrieved.\n\nTool_used: functions.explore_review\n\nRelationship: \n```\nPatient -- Reviews --> Nurse Service Quality Care\nNurse Service Quality Care -- Impacts --> Patient Satisfaction\nNurse Service Quality Care -- Involves --> Competence, Communication, Compassion, Patient Education\n```\n\nResponse:\n- Nurse service quality care is essential for patient outcomes and overall satisfaction.\n- Key aspects of quality care in nursing services include:\n  - **Competence and Knowledge**: Continuous education and training for nurses.\n  - **Compassion and Empathy**: Addressing emotional and psychological needs of patients.\n  - **Communication**: Clear explanations of medical conditions and treatments.\n  - **Patient-Centered Care**: Tailoring care to individual patient needs.\n  - **Team Collaboration**: Working with oth

In [7]:
# List the physicians specializing in Pulmonology at Penang General Hospital.
# do Lam Wah Ee Hospital exist in the database? May I know its contact number
query = "do Lam Wah Ee Hospital exist in the database? May I know its contact number"

response = generate_response(query)

response


2025-01-06 05:26:06.195 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-06 05:26:06.197 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ValueError: Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'checkpoint_ns', 'checkpoint_id']

In [9]:
from langgraph.checkpoint.memory import MemorySaver  # an in-memory checkpointer
from langgraph.prebuilt import create_react_agent

system_message = "You are a helpful assistant."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

memory = MemorySaver()
langgraph_agent_executor = create_react_agent(
    llm, tools, state_modifier=system_message, checkpointer=memory
)

config = {"configurable": {"thread_id": "test-thread"}}
print(
    langgraph_agent_executor.invoke(
        {
            "messages": [
                ("user", "Hi, I'm polly! What's the output of magic_function of 3?")
            ]
        },
        config,
    )["messages"][-1].content
)
print("---")
print(
    langgraph_agent_executor.invoke(
        {"messages": [("user", "Remember my name?")]}, config
    )["messages"][-1].content
)
print("---")
print(
    langgraph_agent_executor.invoke(
        {"messages": [("user", "what was that output again?")]}, config
    )["messages"][-1].content
)

It seems you are asking about a specific function called `magic_function`, but I don't have information about such a function. Could you please clarify what `magic_function` does or provide more context?
---
Yes, your name is Polly! How can I assist you today?
---
I apologize for any confusion earlier, but I don't have access to a function called `magic_function` or its outputs. If you can provide more details about what `magic_function` is supposed to do, I might be able to help you understand or calculate it.


In [ ]:
Neo4jChatMessageHistory(session_id=session_id, graph=graph)